<a href="https://colab.research.google.com/github/nishankhareln/Agent/blob/main/Finetunin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install  torch torchtext sentencepiece pandas tqdm

In [9]:
!pip install transformers

In [10]:
!pip install huggingface_hub

In [13]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.9/520.9 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.34.0 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 1.2.1 which is incompatible.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 1.2.1 which is incompatible.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.34.0 which is incompatible.


In [4]:
!pip install --upgrade datasets


  Using cached huggingface_hub-1.2.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.2.1-py3-none-any.whl (520 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.34.0 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 1.2.1 which is incompatible.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 1.2.1 which is incompatible.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.34.0 which is incompatible.


In [5]:
from datasets import load_dataset,DatasetDict,Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

ImportError: cannot import name 'LastCommitInfo' from 'huggingface_hub.hf_api' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py)

In [2]:
from google.colab import userdata

hf_token = userdata.get('HUGGINGFACE')
print(bool(hf_token))  # True if token is loaded


True


In [3]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv("/content/imputed_dataset.csv")
df.head()

Saving imputed_dataset.csv to imputed_dataset (3).csv


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22.0,59000.0,RENT,123.0,PERSONAL,D,35000.0,16.02,1.0,0.59,Y,3.0
1,21.0,9600.0,OWN,5.0,EDUCATION,B,1000.0,11.14,0.0,0.10,N,2.0
2,25.0,9600.0,MORTGAGE,1.0,MEDICAL,C,5500.0,12.87,1.0,0.57,N,3.0
3,23.0,65500.0,RENT,4.0,MEDICAL,C,35000.0,15.23,1.0,0.53,N,2.0
4,24.0,54400.0,RENT,8.0,MEDICAL,C,35000.0,14.27,1.0,0.55,Y,4.0


In [4]:
def create_prompt(row):
    """Convert a data row into a natural language prompt"""
    prompt = f"""Given the following loan application details:
- Person Age: {row['person_age']} years
- Person Income: ${row['person_income']:,.2f}
- Home Ownership: {row['person_home_ownership']}
- Employment Length: {row['person_emp_length']} years
- Loan Intent: {row['loan_intent']}
- Loan Grade: {row['loan_grade']}
- Loan Amount: ${row['loan_amnt']:,.2f}
- Loan Interest Rate: {row['loan_int_rate']}%
- Loan Percent of Income: {row['loan_percent_income']:.2%}
- Default on File: {row['cb_person_default_on_file']}
- Credit History Length: {row['cb_person_cred_hist_length']} years

Will this loan default?"""

    response = f"The loan will {'default' if row['loan_status'] == 1 else 'not default'}."

    # Format as instruction for LLaMA 3
    formatted_text = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{response}<|eot_id|>"""

    return formatted_text

# Create formatted dataset
print("Creating instruction-formatted dataset...")
df['text'] = df.apply(create_prompt, axis=1)

Creating instruction-formatted dataset...


In [5]:
print("Creating instruction-formatted dataset...")
df['text'] = df.apply(create_prompt, axis=1)

# Display sample
print("\n✓ Sample formatted text:")
print("=" * 80)
print(df['text'].iloc[0])
print("=" * 80)

Creating instruction-formatted dataset...

✓ Sample formatted text:
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Given the following loan application details:
- Person Age: 22.0 years
- Person Income: $59,000.00
- Home Ownership: RENT
- Employment Length: 123.0 years
- Loan Intent: PERSONAL
- Loan Grade: D
- Loan Amount: $35,000.00
- Loan Interest Rate: 16.02%
- Loan Percent of Income: 59.00%
- Default on File: Y
- Credit History Length: 3.0 years

Will this loan default?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The loan will default.<|eot_id|>


In [6]:
from sklearn.model_selection import train_test_split
from datasets import Dataset , DatasetDict

train_df , val_df = train_test_split(df , test_size=0.2 , random_state=42,stratify=df['loan_status'])
print(f"traiing samples:{len(train_df)}")
print(f"validation samples:{len(val_df)}")

traiing samples:26064
validation samples:6517


In [7]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[['text']].reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df[['text']].reset_index(drop=True))

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

print("\n✓ Dataset created successfully!")
print(dataset)


✓ Dataset created successfully!
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 26064
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 6517
    })
})


In [11]:
!pip install huggingface-hub



In [13]:
!pip install transformers

  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.2.1
    Uninstalling huggingface_hub-1.2.1:
      Successfully uninstalled huggingface_hub-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.0 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.17.3 which is incompatible.
datasets 4.4.1 requires huggingface-hub<2.0,>=0.25.0, but you have huggingface-hub 0.17.3 which is incompatible.
accelerate 1.12.0 requires huggingface_hub>=0.21.0, but you have huggingface-hub 0.17.3 which is incompatible.
gradio-client 1.14.0 requires huggingface-hub<2.0,>=0.19.3, but you have huggingface-hub 0.17.3 which is incompatible.
diffusers 0.35.2 requires huggingface-hub>=0.34.0, but 

In [12]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)

ImportError: huggingface-hub>=0.16.4,<1.0 is required for a normal functioning of this module, but found huggingface-hub==1.2.1.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Model configuration
model_name = "meta-llama/Meta-Llama-3-8B"

In [ ]:
!pip install bitsandbytes


In [ ]:
# Configure 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
"""
This is for memory efficiency , faster training, accuracy retention
"""

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HUGGINGFACE')

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    token=hf_token
)


In [ ]:
#load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✓ Tokenizer loaded")

In [ ]:
import bitsandbytes as bnb
print(f"bitsandbytes version: {bnb.__version__}")  # Should print '0.45.2'
from transformers import BitsAndBytesConfig  # No error = success
print("All good—imports working!")

In [ ]:
!pip install bitsandbytes==0.45.2

In [ ]:
# Load model with quantization
from google.colab import userdata
hf_token = userdata.get('HUGGINGFACE')
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    device_map="auto",
    trust_remote_code=True,
)

print("✓ Model loaded successfully")
print(f"Model is on device: {model.device}")

In [ ]:
!pip install --upgrade peft==0.5.0

In [ ]:
!pip install --upgrade peft


In [ ]:
!pip install transformers==4.34.0


In [ ]:
# ============================================================================
# STEP 8: PREPARE MODEL FOR TRAINING WITH LORA
# ============================================================================

print("\n" + "=" * 80)
print("STEP 8: Applying LoRA Configuration")
print("=" * 80)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_pct = 100 * trainable_params / all_params

print(f"✓ LoRA applied successfully!")
print(f"Trainable params: {trainable_params:,}")
print(f"All params: {all_params:,}")
print(f"Trainable %: {trainable_pct:.4f}%")

In [ ]:

# ============================================================================
# STEP 9: CONFIGURE TRAINING ARGUMENTS
# ============================================================================

print("\n" + "=" * 80)
print("STEP 9: Configuring Training")
print("=" * 80)

training_args = TrainingArguments(
    output_dir="./llama3-loan-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Colab can handle 2
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=50,
    eval_steps=50,
    evaluation_strategy="steps",
    save_total_limit=2,
    fp16=True,
    gradient_checkpointing=True,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    report_to="none",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit",  # Memory-efficient optimizer
)

print("✓ Training arguments configured")


In [ ]:
!pip install --upgrade transformers==4.44.2


In [ ]:
# ============================================================================
# STEP 10: INITIALIZE TRAINER
# ============================================================================

print("\n" + "=" * 80)
print("STEP 10: Initializing Trainer")
print("=" * 80)

from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
)

print("✓ Trainer initialized successfully")

In [ ]:
# ============================================================================
# STEP 11: START TRAINING
# ============================================================================

print("\n" + "=" * 80)
print("STEP 11: Starting Training")
print("=" * 80)
print("Training will take approximately 30-60 minutes depending on dataset size...")
print("=" * 80)

# Clear memory before training
torch.cuda.empty_cache()

# Train the model
trainer.train()

print("\n" + "=" * 80)
print("✓ Training Complete!")
print("=" * 80)

In [ ]:
# STEP 12: SAVE THE FINE-TUNED MODEL
# ============================================================================

print("\n" + "=" * 80)
print("STEP 12: Saving Model")
print("=" * 80)

# Save the fine-tuned model
trainer.model.save_pretrained("./llama3-loan-final")
tokenizer.save_pretrained("./llama3-loan-final")

print("✓ Model saved to ./llama3-loan-final")

# Optional: Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')

import shutil
shutil.copytree('./llama3-loan-final', '/content/drive/MyDrive/llama3-loan-final', dirs_exist_ok=True)
print("✓ Model also saved to Google Drive: /MyDrive/llama3-loan-final")

In [ ]:
# ============================================================================
# STEP 13: TEST THE MODEL
# ============================================================================

print("\n" + "=" * 80)
print("STEP 13: Testing the Fine-tuned Model")
print("=" * 80)

# Get a sample from validation set
sample = val_df.iloc[0]

test_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Given the following loan application details:
- Person Age: {sample['person_age']} years
- Person Income: ${sample['person_income']:,.2f}
- Home Ownership: {sample['person_home_ownership']}
- Employment Length: {sample['person_emp_length']} years
- Loan Intent: {sample['loan_intent']}
- Loan Grade: {sample['loan_grade']}
- Loan Amount: ${sample['loan_amnt']:,.2f}
- Loan Interest Rate: {sample['loan_int_rate']}%
- Loan Percent of Income: {sample['loan_percent_income']:.2%}
- Default on File: {sample['cb_person_default_on_file']}
- Credit History Length: {sample['cb_person_cred_hist_length']} years

Will this loan default?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

# Tokenize and generate
model.eval()
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

prediction = tokenizer.decode(outputs[0], skip_special_tokens=False)

print("\n" + "=" * 80)
print("TEST RESULTS")
print("=" * 80)
print("\nModel Prediction:")
print(prediction.split("<|start_header_id|>assistant<|end_header_id|>")[-1].split("<|eot_id|>")[0].strip())
print(f"\nActual Status: {'Default' if sample['loan_status'] == 1 else 'No Default'}")


In [ ]:

# ============================================================================
# STEP 14: INFERENCE FUNCTION FOR NEW PREDICTIONS
# ============================================================================

print("\n" + "=" * 80)
print("STEP 14: Creating Inference Function")
print("=" * 80)

def predict_loan_default(person_age, person_income, person_home_ownership,
                         person_emp_length, loan_intent, loan_grade,
                         loan_amnt, loan_int_rate, loan_percent_income,
                         cb_person_default_on_file, cb_person_cred_hist_length):
    """
    Predict if a loan will default based on application details
    """
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Given the following loan application details:
- Person Age: {person_age} years
- Person Income: ${person_income:,.2f}
- Home Ownership: {person_home_ownership}
- Employment Length: {person_emp_length} years
- Loan Intent: {loan_intent}
- Loan Grade: {loan_grade}
- Loan Amount: ${loan_amnt:,.2f}
- Loan Interest Rate: {loan_int_rate}%
- Loan Percent of Income: {loan_percent_income:.2%}
- Default on File: {cb_person_default_on_file}
- Credit History Length: {cb_person_cred_hist_length} years

Will this loan default?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=False)
    result = prediction.split("<|start_header_id|>assistant<|end_header_id|>")[-1].split("<|eot_id|>")[0].strip()

    return result

print("✓ Inference function created!")

# Example usage
print("\n" + "=" * 80)
print("EXAMPLE PREDICTION")
print("=" * 80)

example_prediction = predict_loan_default(
    person_age=30,
    person_income=50000,
    person_home_ownership="RENT",
    person_emp_length=5,
    loan_intent="PERSONAL",
    loan_grade="B",
    loan_amnt=10000,
    loan_int_rate=11.5,
    loan_percent_income=0.20,
    cb_person_default_on_file="N",
    cb_person_cred_hist_length=5
)

print(f"Prediction: {example_prediction}")

# ============================================================================
# COMPLETION
# ============================================================================

print("\n" + "=" * 80)
print("🎉 ALL STEPS COMPLETED SUCCESSFULLY! 🎉")
print("=" * 80)
print("\n📁 Your model is saved in:")
print("   1. Colab: ./llama3-loan-final")
print("   2. Google Drive: /MyDrive/llama3-loan-final")
print("\n💡 You can now use predict_loan_default() function for predictions!")
print("=" * 80)